In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pylab as plt

import helpfuns as hf
from util import get_checkpoint_path

Data importing

In [2]:
# Default parameters for data loading
DEFAULT_SHUFFLE = True
DATASET_SIZE = 23666
MINI_BATCH_SIZE = 64
EPOCHS = 1
signal_size = 16384

In [3]:
# Directory paths for data loading
tr_data_dir = '../../SC09_Preprocessed/'
tfrecords_fname = 'train.tfrecords'

In [4]:
def _mapper(example_proto):
    features = {
        'signal' : tf.FixedLenSequenceFeature([1], tf.string, allow_missing = True),
        'label': tf.FixedLenSequenceFeature([], tf.string, allow_missing = True)
    }
    example = tf.parse_single_example(example_proto, features)
    
    signal = tf.decode_raw(example['signal'], tf.float32)
    label = tf.reduce_join(example['label'], 0)
        
    return signal, label

    


''' Example code for using the iterator for one epoch
with tf.Session() as sess:
    # Get a random mini-batch 10000 times
    for i in range(10000):
        x = sess.run(xs)
        samples = x[0] # Array with MINI_BATCH_SIZE samples
        labels = x[1] # Label of each sample
'''

def signal_properties(signal):
    """To be implemented"""
    return signal[0].get_shape().as_list()[-1]

def placeholder_factory(signal):
    placeholders = {}
#     signal_size = signal_properties(signal)
    placeholders['True_Signal'] = tf.placeholder(tf.float32,
                                           shape=[batch_size, signal_size, 1],
                                           name='True_Signal')
    placeholders['Fake_Signal'] = tf.placeholder(tf.float32,
                                           shape=[batch_size, signal_size],
                                           name='Fake_Signal')
    placeholders['output'] = lambda: tf.placeholder(tf.int16, shape=(None, 1))
    placeholders['is_training']= tf.placeholder(tf.bool, name="is_training")
    return placeholders


Define Models

In [5]:
   
def PhaseShuffle(net, scope=None):
    """randomly perturbs the phase of each layer’s activations 
    by −2 to 2 samples before input to the next layer. 
    We use reflection padding to fill in the the missing samples"""
    rand_phase = tf.random_uniform([], 
                              minval=-2, 
                              maxval=2 + 1, 
                              dtype=tf.int32,
                              name="rand_sampling")
    pading_scheme = [[0, 0], [ tf.maximum(rand_phase, 0), tf.maximum(-rand_phase, 0)], [0, 0]]
    org_dims = net.get_shape().as_list()
    net = tf.pad(net,
                 paddings=pading_scheme,
                 mode='REFLECT',
                 name="refl_pading")[:, tf.maximum(-rand_phase, 0):tf.maximum(-rand_phase, 0)+org_dims[1]]
    net.set_shape(org_dims)
    net = tf.identity(net, name=scope+"_PhaseShuffle")
    return net

# Deconvolution Opperation
def conv1d_transpose(inputs,filters,kernel_width=25,stride=4,activation=tf.nn.relu,batchnorm = False,is_training=True,scope=None):
    
    net = tf.layers.conv2d_transpose(
        tf.expand_dims(inputs, axis=1),
        filters=filters,
        kernel_size=(1, kernel_width),
        strides=(1, stride),
        padding='SAME',
        activation=None,
        name=scope+"_conv1d")[:, 0]


    if batchnorm:
        net = tf.layers.batch_normalization(net, training=is_training, fused=True, name=scope+"_BN")
        net = activation(net, name=scope+"_act")
    else:
        net = activation(net, name=scope+"_actv")
    return net

def add_special_layers(net,scope=None, is_training=True, batchnorm=True, activation=None, shuffle=True):
    if shuffle:
        net = PhaseShuffle(net, scope=scope)
    if batchnorm:
        net = tf.layers.batch_normalization(net, training=is_training, fused=True, name=scope+"_BN")
    if activation != None:
        net = activation(net, name=scope+"_actv")
    return net

In [6]:
def generator(placeholders, add_filer=False):
    with tf.variable_scope('Generator'):
            is_training = placeholders['is_training']
            net = tf.layers.dense(placeholders['Fake_Signal'],
                                  units=16384,
                                  kernel_initializer=None,
                                  name="fc1")
            net = tf.layers.batch_normalization(net, training=is_training, fused=True, name="fc1_BN")
            net = tf.nn.relu(net, name="fc1_ReLu")
            net = tf.reshape(net, [batch_size, 16, 1024], name="fc1_rs")
            net = conv1d_transpose(net,
                                   filters=512,
                                   kernel_width=25,
                                   stride=4, 
                                   batchnorm = True,
                                   is_training=placeholders['is_training'],
                                   scope="tc1")
            net = conv1d_transpose(net,
                                   filters=256,
                                   kernel_width=25,
                                   stride=4, 
                                   batchnorm = True,
                                   is_training=placeholders['is_training'],
                                   activation=tf.nn.relu,
                                   scope="tc2")
            net = conv1d_transpose(net,
                                   filters=128,
                                   kernel_width=25,
                                   stride=4, 
                                   batchnorm = True,
                                   activation=tf.nn.relu,
                                   is_training=placeholders['is_training'],
                                   scope="tc3")
            net = conv1d_transpose(net,
                                   filters=64,
                                   kernel_width=25,
                                   stride=4, 
                                   batchnorm = True,
                                   activation=tf.nn.relu,
                                   is_training=placeholders['is_training'],
                                   scope="tc4")
            net = conv1d_transpose(net,
                                   filters=1,
                                   kernel_width=25,
                                   stride=4, 
                                   batchnorm = True,
                                   activation=tf.nn.tanh,
                                   is_training=placeholders['is_training'],
                                   scope="tc5")  
            
            if add_filer:
                net = tf.layers.conv1d(net, 
                                       filters=1, 
                                       kernel_size=512, 
                                       use_bias=False, 
                                       padding='SAME',
                                      name="add_G_filer")
                
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                net = tf.identity(net, name="Generator_OUTPUT")
                
            return net

def discriminator(inputs, is_training=True):
    with tf.variable_scope('Discriminator'):
#         inputs = tf.layers.flatten(inputs)
#         inputs = tf.expand_dims(inputs, 1)
#         net = tf.reshape(inputs, [inputs.get_shape().as_list()[0], 
#                                   inputs.get_shape().as_list()[2], 1], 
#                          name="dum_reshape")
        net = tf.layers.conv1d(inputs,
                               filters=64, 
                               kernel_size=25, 
                               strides=4, 
                               padding='SAME',
                               kernel_initializer=None,
                               name="conv1")
        net = add_special_layers(net, 
                                 scope="conv1", 
                                 is_training=is_training, 
                                 batchnorm=True, 
                                 activation=tf.nn.leaky_relu, 
                                 shuffle=True)
        net = tf.layers.conv1d(inputs,
                               filters=128, 
                               kernel_size=25, 
                               strides=4, 
                               padding='SAME',
                               kernel_initializer=None,
                               name="conv2")
        net = add_special_layers(net, 
                                 scope="conv2", 
                                 is_training=is_training, 
                                 batchnorm=True, 
                                 activation=tf.nn.leaky_relu, 
                                 shuffle=True)
        net = tf.layers.conv1d(inputs,
                               filters=256, 
                               kernel_size=25, 
                               strides=4, 
                               padding='SAME',
                               kernel_initializer=None,
                               name="conv3")
        net = add_special_layers(net, 
                                 scope="conv3", 
                                 is_training=is_training, 
                                 batchnorm=True, 
                                 activation=tf.nn.leaky_relu, 
                                 shuffle=True)
        net = tf.layers.conv1d(inputs,
                               filters=512, 
                               kernel_size=25, 
                               strides=4, 
                               padding='SAME',
                               kernel_initializer=None,
                               name="conv4")
        net = add_special_layers(net, 
                                 scope="conv4", 
                                 is_training=is_training, 
                                 batchnorm=True, 
                                 activation=tf.nn.leaky_relu, 
                                 shuffle=True)
        net = tf.layers.conv1d(inputs,
                               filters=1024, 
                               kernel_size=25, 
                               strides=4, 
                               padding='SAME',
                               kernel_initializer=None,
                               name="conv5")
        net = add_special_layers(net, 
                                 scope="conv5", 
                                 is_training=is_training, 
                                 batchnorm=True, 
                                 activation=tf.nn.leaky_relu, 
                                 shuffle=False)
        net = tf.layers.flatten(net,name="flatten")
        net = tf.layers.dense(net, 
                              units=1,
                              kernel_initializer=None,
                              name="fc_out")[:, 0]
        net = tf.identity(net, name="Discriminator_OUTPUT")
        
        return net

Define Gan Graph

In [7]:
def create_model_graph(graph,sess,placeholders):
    # Get Generator
    Generator_model = generator(placeholders, add_filer=False)
    G_train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='Generator')
    # Get Real Discriminator
    with tf.name_scope('Real_D'), tf.variable_scope('D_train_vars'):
        Real_D_model = discriminator(placeholders['True_Signal'], is_training=True)
    D_train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='D_train_vars')
    # Get Fake Discriminator with resue
    with tf.name_scope('Fake_D'), tf.variable_scope('D_train_vars', reuse=True):
        Fake_D_model = discriminator(Generator_model, is_training=True)
        
    # Define loss connectivity (Used opt settings as in the paper)
    Generator_loss = -tf.reduce_mean(Fake_D_model)
    Discriminator_loss = tf.reduce_mean(Fake_D_model) - tf.reduce_mean(Real_D_model)

    alpha = tf.random_uniform(shape=[batch_size, 1, 1], minval=0., maxval=1.)
    differences = Generator_model - placeholders['True_Signal']
    interpolate = placeholders['True_Signal'] + (alpha * differences)
    with tf.name_scope('Interpolator'), tf.variable_scope('D_train_vars', reuse=True):
        D_interpolator = discriminator(interpolate, is_training=True)

    LAMBDA = 10
    gradients = tf.gradients(D_interpolator, [interpolate])[0]
    slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), reduction_indices=[1, 2]))
    gradient_penalty = tf.reduce_mean((slopes - 1.) ** 2.)
    Discriminator_loss += LAMBDA * gradient_penalty
    
    # Define Optimizers
    G_opt= tf.train.AdamOptimizer(1e-4, beta1=0.5,beta2=0.9).minimize(Generator_loss, 
                                                    var_list=G_train_vars, 
                                                    global_step=tf.train.get_or_create_global_step())
    D_opt= tf.train.AdamOptimizer(1e-4, beta1=0.5,beta2=0.9).minimize(Discriminator_loss, 
                                                    var_list=D_train_vars)
    
    return Generator_model, G_opt, D_opt

Training Session

In [8]:
# To be changed!
epochs= 1

batch_size = 12
input_dim = 100

dataset = tf.data.TFRecordDataset([tr_data_dir + tfrecords_fname])
# Repeat dataset indefinitely
dataset = dataset.repeat()
# Shuffle the dataset
dataset = dataset.shuffle(buffer_size = DATASET_SIZE)
# Extract features from inputs
dataset = dataset.map(_mapper)
# Drop remainder samples so that all mini batches have the same size
dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(batch_size))
# Get one shot iterator to iterate over mini-batches
xs = dataset.make_one_shot_iterator().get_next() 

graph = tf.Graph()
sess = tf.Session(graph=graph)
with graph.as_default():
    with tf.Session() as sess: 
        # create placeholders
        placeholders = placeholder_factory(xs)
#         # Create Real Input
#         Real_data = dataset.make_one_shot_iterator().get_next()
#         # Create Generator input
#         Gen_input = tf.random_uniform([batch_size,1,1, input_dim], minval=-1.0, maxval=1.0)
        # Create GAN graph
        Generator_model, G_opt, D_opt = create_model_graph(graph, sess, placeholders)
        # initialize vars
        sess.run(tf.global_variables_initializer())

        # create savers
        saver = tf.train.Saver()
        Generator_saver = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Generator')
        save_path = get_checkpoint_path("Full_Model")
        Generator_save_path = get_checkpoint_path("Generator_Model")

        # initialize training
        ticks = 0
        for epoch in range(epochs):
            for i in range(100):
                Gen_input = tf.random_uniform([batch_size,1,1, input_dim], minval=-1.0, maxval=1.0)
                Real_data = dataset.make_one_shot_iterator().get_next()
                ticks +=1
                fed_dict = {
                    placeholders['True_Signal']: Real_data[0],
                    placeholders['Fake_Signal']: Gen_input,
                    placeholders['is_training']: True
                }

                Gloss, _, _ = sess.run([Generator_model, G_opt, D_opt], feed_dict=fed_dict)
                print("\rEpoch: %i - batch %i - iteration %i - G_loss %f" % (epoch, i, ticks, Gloss))


        saver.save(sess, save_path)
        move(os.path.join(os.getcwd(), "checkpoints/") + "checkpoint", save_path)
        Generator_saver.save(sess, save_path)
        move(os.path.join(os.getcwd(), "checkpoints/") + "checkpoint", Generator_save_path)


tf.reset_default_graph()
sess.close()


Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


ResourceExhaustedError: OOM when allocating tensor with shape[16384,16384] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: Generator/fc1/kernel/Adam/Assign = Assign[T=DT_FLOAT, _class=["loc:@Generator/fc1/kernel"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Generator/fc1/kernel/Adam, Generator/fc1/kernel/Adam/Initializer/zeros)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'Generator/fc1/kernel/Adam/Assign', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mats/.local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/mats/.local/lib/python3.

Run results

In [ ]:
tf.reset_default_graph()
sess.close()
# graph = tf.Graph()
# sess = tf.Session(graph=graph)
# with graph.as_default():
#     sess.run(dataset.make_one_shot_iterator().get_next())

In [ ]:
tf.reset_default_graph()
sess.close()